In [ ]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import time
warnings.filterwarnings('ignore')
from importlib import reload
#reload(gpd)

# After Obtaining Featurizations from MOSAIKS API

In [ ]:
TILES_FOLDER = '/data/mosaiks/sampled_tiles/'
FEATURES_FOLDER = '/data/mosaiks/features/replication/'
FEATURES_FOLDER_1_TILE = '/data/mosaiks/replication/features_1_tile_per_region/'

In [ ]:
# Read raw mosaiks features
def get_features(FEATURES_INFOLDER, SAMPLE_FNAME):
    mosaiks = []
    for fname in os.listdir(FEATURES_INFOLDER):
        if fname[-4:] == '.csv':
            print(fname)
            mosaiks.append(pd.read_csv(FEATURES_INFOLDER + '/' + fname))

    if 'Lat' in mosaiks[0].keys():
        mosaiks = pd.concat(mosaiks).drop_duplicates(subset=['Lat', 'Lon'])
    else:
        mosaiks = pd.concat(mosaiks).drop_duplicates(subset=['Latitude', 'Longitude'])
    mosaiks.columns = ['Latitude', 'Longitude'] + ['Feature' + str(i) for i in range(len(mosaiks.columns) - 2)]

    # Merge mosaiks features to original sample file
    sample = pd.read_csv(SAMPLE_FNAME)
    mosaiks = mosaiks.merge(sample, how='inner', on=['Latitude', 'Longitude'])
    
    return mosaiks

In [ ]:
dhs_countries = ['colombia',  'honduras', 'indonesia', 'nigeria', 'kenya', 'peru', 'philippines']
countries = dhs_countries + ['us', 'mexico']
# india is done in a separate notebook

seed = 0
rs = np.random.RandomState(seed)
    
for country in countries:
    if country.lower() == 'us':
        SAMPLE_FNAME = TILES_FOLDER + 'us/sampled_tiles.csv'
        SHAPEFILE_IDS = ['PUMA','State']
        OUT_FNAME = FEATURES_FOLDER_1_TILE + 'mosaiks_features_by_puma_us.csv'
        FEATURES_INFOLDER = TILES_FOLDER + 'us/featurized/'

    elif country.lower() == 'mexico':
        # RUN THIS CELL FOR MEXICO MUNICIPALITIES
        FEATURES_INFOLDER = TILES_FOLDER + 'mexico/featurized/'
        SAMPLE_FNAME = TILES_FOLDER + 'mexico/sampled_tiles.csv'
        SHAPEFILE_IDS = ['municipality']
        OUT_FNAME = FEATURES_FOLDER_1_TILE + 'mosaiks_features_by_municipality_mexico.csv'

    else:
        dhs_country = country
        FEATURES_INFOLDER = TILES_FOLDER + 'dhs/' + dhs_country + '/featurized/'
        SAMPLE_FNAME = TILES_FOLDER + 'dhs/' + dhs_country + '/sampled_tiles.csv'
        SHAPEFILE_IDS = ['cluster']
        OUT_FNAME = FEATURES_FOLDER_1_TILE + 'dhs/mosaiks_features_by_cluster_' + dhs_country + '.csv'
    
    %time feats = get_features(FEATURES_INFOLDER, SAMPLE_FNAME)
        
    # take a sample of one from each location    
    sample = feats.groupby(SHAPEFILE_IDS, as_index=False).sample(n=1, replace=False, random_state=rs)
    
    print(f'writing to {OUT_FNAME}')
    sample.to_csv(OUT_FNAME)